# Graph mapping

## Subclassing `GraphModule`

In [2]:
import rockpool.graph as rg
from dataclasses import dataclass

@dataclass(eq = False)
class MyGraphModule(rg.GraphModule):
    # - Define parameters as for any dataclass
    param1: float
    param2: int
    param3: list

In [3]:
class MyGraphModule(MyGraphModule):
    # - Any initialisation checks can be performed a __post_init__ method
    def __post_init__(self, *args, **kwargs):
        # - You *must* call super().__post_init__()
        super().__post_init__(*args, **kwargs)
        
        if param1 < param2:
            raise ValueError('param1 must be > param2')

## Transforming `GraphModule` s

## Creating a mapper

## DRC checks